# ☁️ FinOps CUD Analysis and Reporting

**Author:** andrewanolasco@
**Version:** v1.0.0

---

### Overview
Welcome to the FinOps CUD Analysis Platform! This notebook is your central hub for analyzing your Google Cloud spend, identifying cost-saving opportunities, and generating executive-level reports. 

**This single notebook allows you to:**
1.  **Run a full, automated analysis** to generate a PDF report and interactive dashboard.
2.  **Perform an interactive deep-dive** into your data using AI-powered tools.
3.  **Fetch the latest CUD pricing** directly from the Google Cloud Billing API.

## ⚙️ 1. Setup and Configuration

This section handles all the necessary setup, including installing dependencies, importing libraries, and loading your configuration.

### 1.1. Install Dependencies

In [ ]:
!pip install -q -r ../requirements.txt

### 1.2. Imports and Path Setup

In [ ]:
import loggingimport sysfrom pathlib import Path# Add the src directory to the path to allow importing our modulessys.path.append(str(Path.cwd().parent / 'src'))from finops_analysis_platform.config_manager import ConfigManagerfrom finops_analysis_platform.data_loader import load_data_from_configfrom finops_analysis_platform.core import CUDAnalyzerfrom finops_analysis_platform.reporting import PDFReportGenerator, create_dashboardlogging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### 1.3. Load Configuration and Data

In [ ]:
# @markdown Specify the path to your configuration file and an optional output name for the report.CONFIG_PATH = 'config.yaml' # @param {type:"string"}OUTPUT_REPORT_NAME = '' # @param {type:"string"}

In [ ]:
logging.info(f"Loading configuration from: {CONFIG_PATH}")config_manager = ConfigManager(config_path=CONFIG_PATH)data = load_data_from_config(config_manager)billing_data = data.get('billing')recommendations_data = data.get('recommendations')logging.info("Data loaded successfully.")

## 📊 2. Automated Analysis and Reporting

Run the cells in this section to perform a full, end-to-end analysis. It will generate a comprehensive PDF report and an interactive dashboard summarizing your CUD and Active Assist savings opportunities.

In [ ]:
if billing_data is not None and not billing_data.empty:    logging.info("Starting full analysis...")    analyzer = CUDAnalyzer(config_manager=config_manager, billing_data=billing_data, recommendations_data=recommendations_data)    analysis = analyzer.generate_comprehensive_analysis()    logging.info("Full analysis complete.")    # Generate PDF Report    logging.info("Generating PDF report...")    pdf_generator = PDFReportGenerator(config_manager=config_manager)    report_filename = pdf_generator.generate_report(analysis, filename=OUTPUT_REPORT_NAME)    logging.info(f"PDF report generated: {report_filename}")    # Create and display Interactive Dashboard    logging.info("Creating interactive dashboard...")    dashboard = create_dashboard(analysis, config_manager)    dashboard.show()else:    logging.warning("Billing data is empty. Skipping analysis and report generation.")

## 🔬 3. Interactive Deep-Dive with AI (Optional)

This section provides advanced, interactive tools for asking ad-hoc questions about your data using the Gemini AI.

### 3.1. AI-Powered Data Analysis

In [ ]:
import ipywidgets as widgetsfrom IPython.display import Markdown, display, clear_outputfrom finops_analysis_platform.gemini_service import generate_contentcode_prompt_input = widgets.Textarea(    value='Analyze the billing data in `billing_df`. What are the top 3 services by cost? Display the result as a table.',    placeholder='Type your prompt here... e.g., Show me the total cost per project.',    description='Prompt:',    layout={'width': '100%', 'height': '100px'})code_run_button = widgets.Button(description="Run AI Analysis")code_output_area = widgets.Output()def run_code_execution(b):    with code_output_area:        clear_output()        print("Generating and executing code...")        # Note: Code Execution is a powerful feature of the Gemini API, not Vertex AI        # This demonstrates the concept; a full implementation would require the google-generativeai library        # configured for the Gemini API with a separate API key.        display(Markdown("**Note:** This is a conceptual demonstration. Full code execution requires the Gemini API."))code_run_button.on_click(run_code_execution)display(code_prompt_input, code_run_button, code_output_area)

##  utility 4. CUD Price Fetcher (Optional)

Use this utility to fetch the latest pricing for both old and new CUD models directly from the Google Cloud Billing API.

In [ ]:
import osimport pandas as pdfrom getpass import getpassfrom finops_analysis_platform.gcp_pricing import GcpSkuFetcher# --- Secure API Key Handling ---api_key = os.getenv("CLOUD_BILLING_API_KEY")if not api_key:    try:        api_key = getpass('Enter your Cloud Billing API Key: ')    except NameError: # Fallback for non-interactive environments        api_key = Noneif not api_key:    print("ERROR: CLOUD_BILLING_API_KEY not found as an environment variable or provided via prompt.")else:    # Initialize the fetcher with your API key    sku_fetcher = GcpSkuFetcher(api_key=api_key)        # Fetch and analyze CUD prices    cud_data = sku_fetcher.analyze_cud_prices()        if cud_data:        # For better analysis, load the data into a pandas DataFrame        df_cud_prices = pd.json_normalize(cud_data, record_path=['pricing'], meta=['description', 'model_type', 'sku_id'])        print("
--- CUD Pricing Analysis ---")        display(df_cud_prices[['model_type', 'description', 'sku_id', 'price', 'currency', 'usage_unit']])